In [49]:
import pandas as pd
import time
import random
import json
from datetime import datetime
from pytz import timezone

# scrapping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import bs4
import requests
from requests.exceptions import RequestException



from CHN.general import Scrapping


import warnings
warnings.filterwarnings('ignore')

In [50]:
scrap = Scrapping()

In [51]:
# ubicacion de chrome driver y la url que queremos utilizar
PATH_TO_DRIVER = 'C:/chromedriver-win32/chromedriver-win32/chromedriver.exe'
url = 'https://www.plazapublica.com.gt/'

In [52]:
# creamos el objeto service
service = Service(PATH_TO_DRIVER)

# se inicializa el navegador con el servicio
driver = webdriver.Chrome(service=service)

In [53]:
driver.get(url)

WebDriverWait(driver, 10)

<selenium.webdriver.support.wait.WebDriverWait (session="8b62b4d5e1a4fb59206b7a465722b7d1")>

In [54]:
try:
    buscar_icono = driver.find_element(By.CSS_SELECTOR, '.top-buscar a')
    buscar_icono.click()
    time.sleep(3)
except:
    print('No fue necesario hacer click')

In [55]:
# Localizar el campo de búsqueda dentro de Google Custom Search
search_box = driver.find_element(By.CSS_SELECTOR, "input.gsc-input")

# Escribir el término de búsqueda y presionar Enter
search_box.send_keys("bernardo arevalo")
search_box.send_keys(Keys.RETURN)

In [56]:
time.sleep(5)

# extraemos el html actual
html_actual = driver.page_source

print(type(html_actual))

driver.quit()

<class 'str'>


In [57]:
soup = BeautifulSoup(html_actual, 'html.parser')

with open('../data/plazapublica/busqueda_selenium_v2.html', 'w') as home:
    home.write(soup.prettify())

In [58]:
def extraer_articulos(soup : bs4.BeautifulSoup) -> dict:
    title_tag = soup.select_one('.gs-title a')
    url_tag = title_tag['href'] if title_tag else None
    title = title_tag.get_text(strip=True) if title_tag else None

    snippet_tag = soup.select_one('.gs-snippet')
    snippet = snippet_tag.get_text(strip=True) if snippet_tag else None

    source_tag = soup.select_one('.gs-visibleUrl-short')
    source = source_tag.get_text(strip=True) if source_tag else None

    return {
        "title": title,
        "url": url_tag,
        "snippet": snippet,
        "source": source
    }


In [59]:
articles = [extraer_articulos(article) for article in soup.select('div.gsc-webResult')]

In [60]:
with open('../data/plazapublica/export_bernardo_v2.json', 'w', encoding='utf-8') as ex:
    json.dump({"plaza publica": [{"page": 1, "articulos": articles}]}, ex, ensure_ascii=False, indent=4)